In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
csv_filename = f'DODFCorpus_contratos_licitacoes_concordancia.csv'
df = pd.read_csv(csv_filename, index_col=[0])
df

,id_dodf,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,672,EXTRATO_CONTRATO,1,anotador_59684932,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
1,672,EXTRATO_CONTRATO,2,anotador_69243565,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
2,672,EXTRATO_CONTRATO,3,anotador_72589596,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
3,672,processo_gdf,5,anotador_69243565,53,15,001-000853/2019
4,672,processo_gdf,6,anotador_72589596,53,15,001-000853/2019
...,...,...,...,...,...,...,...
27738,638,objeto_licitacao,3737,anotador_37219154,210322,367,Registro de Preços para contratação de empresa...
27739,638,orgao_licitante,3738,anotador_37219154,210631,58,Secretaria de Estado do Trabalho Distrito Fede...
27740,638,orgao_licitante,3739,anotador_27589498,210631,58,Secretaria de Estado do Trabalho Distrito Fede...
27741,638,nome_responsavel,3740,anotador_37219154,210768,25,JUNILMA OLIVEIRA FERREIRA


In [3]:
# Divide o dataframe for DODF

id_dodfs = df['id_dodf'].unique()

# Chave = id_dodf, valor = dataframe com as anotacoes do dodf
dodf_dataframes = {}

for id_dodf in id_dodfs:
    dodf_dataframes[id_dodf] = df[df['id_dodf'] == id_dodf]

dodf_dataframes[id_dodfs[0]]

,id_dodf,tipo_ent,id_ent,anotador_ent,offset,length,texto
0,672,EXTRATO_CONTRATO,1,anotador_59684932,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
1,672,EXTRATO_CONTRATO,2,anotador_69243565,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
2,672,EXTRATO_CONTRATO,3,anotador_72589596,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....
3,672,processo_gdf,5,anotador_69243565,53,15,001-000853/2019
4,672,processo_gdf,6,anotador_72589596,53,15,001-000853/2019
...,...,...,...,...,...,...,...
2270,672,vigencia_contrato,2375,anotador_59684932,166035,46,12 (doze) meses a contar da data de assinatura
2271,672,vigencia_contrato,2376,anotador_69243565,166035,46,12 (doze) meses a contar da data de assinatura
2272,672,data_assinatura_contrato,2377,anotador_72589596,166103,10,25/07/2019
2273,672,data_assinatura_contrato,2378,anotador_59684932,166103,10,25/07/2019


Para cada DODF, são agrupadas as anotações referentes a uma mesma entidade feias por anotadores diferetes.
Os critérios para determinar se uma anotação é referente à mesma entidade que uma anotação tomada como base (critérios de pertinência) são:

1. O offset de início das duas anotações são iguais com uma margem de caracteres de 10% do tamanho da anotação base;

2. Os tamanhos das duas anotações são iguais com uma margem de 10%.

In [4]:
# Agrupa as anotacoes pertinentes a uma mesma entidade

# Chave = id_dodf
# Valor = lista de conjuntos de anotacoes que se referem a uma mesma entidade
dodf_annotation_groups = {}

for id_dodf in id_dodfs:
    df = dodf_dataframes[id_dodf]
    dodf_annotation_groups[id_dodf] = []

    # Chave = indice de uma anotacao do dataframe
    # Valor = lista de indices de anotacoes que satisfazem os criterios de pertinencia
    # em relacao a anotacao da chave.
    annotation_groups_redundant = {}
    for index in df.index:
        base_annotation = df.loc[index]
        base_offset = base_annotation['offset']
        base_length = base_annotation['length']
        
        # Primeiro criterio
        # TODO aplicar o bonus de tolerancia para entidades menores que nem para length
        # TODO testar com +/- (0.1+13/base_length)*base_length
        # esse valor abrange o caso de "prestação de serviços" vs "serviços"
        min_offset  = base_offset-(0.1+13/base_length)*base_length
        max_offset  = base_offset+(0.1+13/base_length)*base_length
        annotations = df[(df['offset'] > min_offset) & (df['offset'] < max_offset)]

        # Segundo criterio
        # TODO testar 23 no lugar do 6
        # esse valor abrange o caso de "prestação de serviços" vs "serviços"
        min_length  = base_length-(0.1+23/base_length)*base_length
        max_length  = base_length+(0.1+23/base_length)*base_length
        annotations = annotations[(annotations['length'] > min_length) & (annotations['length'] < max_length)]
        
        annotation_groups_redundant[index] = list(annotations.index)

    # Para remover os grupos redundantes, converte-se o dicionario para uma lista de sets.
    # Cada set contem o indice usado como chave e os indices da sua lista de valores.
    for key_annotation, annotation_list in annotation_groups_redundant.items():
        annotation_list.append(key_annotation)
        dodf_annotation_groups[id_dodf].append(frozenset(annotation_list))

    dodf_annotation_groups[id_dodf] = set(dodf_annotation_groups[id_dodf])


dodf_annotation_groups[id_dodfs[1]]

{frozenset({4333, 4334}),
 frozenset({3717}),
 frozenset({2416, 2417, 2420}),
 frozenset({3313}),
 frozenset({3287}),
 frozenset({2909, 2910, 2911}),
 frozenset({2294, 2295, 2296}),
 frozenset({3618}),
 frozenset({3729, 3730}),
 frozenset({3871, 3872, 3873}),
 frozenset({4132, 4133}),
 frozenset({4160, 4161}),
 frozenset({4269, 4271, 4273}),
 frozenset({4365, 4366, 4367}),
 frozenset({3288, 3289}),
 frozenset({4404, 4405}),
 frozenset({3510, 3511, 3512}),
 frozenset({4320, 4321}),
 frozenset({3435}),
 frozenset({3436}),
 frozenset({4454, 4455}),
 frozenset({3126, 3127}),
 frozenset({3005, 3006, 3007}),
 frozenset({3437}),
 frozenset({4294, 4295}),
 frozenset({2834, 2835, 2836}),
 frozenset({2694, 2695}),
 frozenset({4140, 4141}),
 frozenset({2421, 2422}),
 frozenset({2792, 2793, 2794}),
 frozenset({2933, 2934, 2935}),
 frozenset({3981, 3982}),
 frozenset({2938, 2939, 2940}),
 frozenset({3892, 3893}),
 frozenset({2413, 2414, 2415}),
 frozenset({3962, 3963, 3964}),
 frozenset({4369, 4370

# Métricas

In [5]:
# Jaccard
def jaccard_char(str1, str2):
    diferenca_char = 0
    # string 2 maior que string 1
    if len(str2) > len(str1):
        inicio = str2.find(str1)
        # teste substring
        if inicio == -1:
            agreement = 0
            return agreement
        else:
            idx_menor = 0
            for idx, _ in enumerate(str2):
                try:
                    # shift na string até encontrar o caractere de inicio
                    if idx < inicio:
                        diferenca_char += 1
                        # contador para shift-reverso
                        idx_menor +=1
                    else:
                        # strings 'alinhadas', diferença char a char
                        if str2[idx] != str1[idx-idx_menor]:
                            diferenca_char += 1
                # acabamos de percorrer str1; computamos a diferença remanescente em relação à str2
                except:
                    dif_reman = len(str2) - idx
                    diferenca_char += dif_reman
                    break
            # computamos o agreement usando jaccard
            agreement = abs(len(str2) - diferenca_char)/len(str2)
            
    # string 1 maior que string 2
    elif len(str1) > len(str2):
        # teste substring
        inicio = str1.find(str2)
        if inicio == -1:
            agreement = 0
            return agreement
        else:
            idx_menor = 0
            for idx, _ in enumerate(str1):
                try:
                    # shift na string até encontrar o caractere de inicio
                    if idx < inicio:
                        diferenca_char += 1
                        # contador para shift-reverso
                        idx_menor +=1
                    else:
                        # strings 'alinhadas', diferença char a char
                        if str1[idx] != str2[idx-idx_menor]:
                            diferenca_char += 1
                # acabamos de percorrer str2; computamos a diferença remanescente em relação à str1
                except:
                    dif_reman = len(str1) - idx
                    diferenca_char += dif_reman
                    break
            # computamos o agreement usando jaccard
            agreement = abs(len(str1) - diferenca_char)/len(str1)
            
    # strings de mesmo tamanho
    else:
        # teste subtring
        inicio = str1.find(str2)
        if inicio == -1:
            agreement = 0
            return agreement
        # str1 e str2 são a mesma string
        elif inicio == 0:
            agreement = 1
        else:
            # strings alinhadas, diferença char a char
            for idx, _ in enumerate(str1):
                if str1[idx] != str2[idx]:
                    diferenca_char += 1
            total_dif = abs(len(str1) - diferenca_char)/len(str1)
            print(f'total_dif caso 3 = {total_dif}')
            agreement = 1 - total_dif
    
    return agreement

In [6]:
# Cosine similarity
def cos_sim(vec1, vec2):
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    cos = dot/(norm1 * norm2)
    return cos

In [7]:
# Edit distance
def edit_distance(str1, str2):
    if len(str1) > len(str2):
        str1, str2 = str2, str1

    distances = range(len(str1) + 1)
    for i2, char2 in enumerate(str2):
        distances_ = [i2+1]
        for i1, char1 in enumerate(str1):
            if char1 == char2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

# Estruturas auxiliares para similaridade cosseno: TFIDF e Word2Vec

_*Similaridade cosseno com Word2Vec não implementada até o momento_

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = []
for df in dodf_dataframes.values():
    corpus.extend(df['texto'].values)

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(corpus)

dodf_annotation_tfidfs = {}
for id_dodf, df in dodf_dataframes.items():
    tfidfs = tfidf_vectorizer.transform(df['texto'].values).toarray()
    dodf_annotation_tfidfs[id_dodf] = {index: tfidf for index, tfidf in list(zip(df.index, tfidfs))}



# Calcula as métricas para cada anotação

In [9]:

for id_dodf, annotation_groups in dodf_annotation_groups.items():
    df = dodf_dataframes[id_dodf]
    
    if 'edit_distance' not in df:
        df.insert(len(df.columns), 'edit_distance', np.nan)
    if 'jaccard_index' not in df:
        df.insert(len(df.columns), 'jaccard_index', np.nan)
    if 'cos_sim_tfidf' not in df:
        df.insert(len(df.columns), 'cos_sim_tfidf', np.nan)
    if 'cos_sim_word2vec' not in df:
        df.insert(len(df.columns), 'cos_sim_word2vec', np.nan)
    
    for group in annotation_groups:
        pairs = list(itertools.combinations(group, 2))
        group_edit_distances    = []
        group_jaccard_indexes   = []
        group_cos_sims_tfidf    = []
        group_cos_sims_word2vec = []
        
        for anno_index1, anno_index2 in pairs:
            anno1     = df.loc[anno_index1]['texto']
            anno2     = df.loc[anno_index2]['texto']
            tfidf1    = dodf_annotation_tfidfs[id_dodf][anno_index1]
            tfidf2    = dodf_annotation_tfidfs[id_dodf][anno_index2]
            # word2vec1 = get_word2vec(anno1)
            # word2vec2 = get_word2vec(anno2)
            
            group_edit_distances.append(edit_distance(anno1, anno2))
            group_jaccard_indexes.append(jaccard_char(anno1, anno2))
            group_cos_sims_tfidf.append(cos_sim(tfidf1, tfidf2))
            # group_cos_sims_word2vec.append(cos_sim(word2vec1, word2vec2))

        for anno_index in group:
            if group_edit_distances:
                df.loc[anno_index, 'edit_distance']    = np.nanmean(group_edit_distances)
            if group_jaccard_indexes:
                df.loc[anno_index, 'jaccard_index']    = np.nanmean(group_jaccard_indexes)
            if group_cos_sims_tfidf:
                df.loc[anno_index, 'cos_sim_tfidf']    = np.nanmean(group_cos_sims_tfidf)
            # if group_cos_sims_word2vec:
                # df.loc[anno_index, 'cos_sim_word2vec'] = np.nanmean(group_cos_sims_word2vec)
            

dodf_dataframes[672]

/tmp/ipykernel_17277/1194147263.py:6: RuntimeWarning: invalid value encountered in double_scalars
  cos = dot/(norm1 * norm2)
/tmp/ipykernel_17277/1194147263.py:6: RuntimeWarning: invalid value encountered in double_scalars
  cos = dot/(norm1 * norm2)
/tmp/ipykernel_17277/695528714.py:39: RuntimeWarning: Mean of empty slice
  df.loc[anno_index, 'cos_sim_tfidf']    = np.nanmean(group_cos_sims_tfidf)
/tmp/ipykernel_17277/1194147263.py:6: RuntimeWarning: invalid value encountered in double_scalars
  cos = dot/(norm1 * norm2)
/tmp/ipykernel_17277/695528714.py:39: RuntimeWarning: Mean of empty slice
  df.loc[anno_index, 'cos_sim_tfidf']    = np.nanmean(group_cos_sims_tfidf)


,id_dodf,tipo_ent,id_ent,anotador_ent,offset,length,texto,edit_distance,jaccard_index,cos_sim_tfidf,cos_sim_word2vec
0,672,EXTRATO_CONTRATO,1,anotador_59684932,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.0,1.0,NaN
1,672,EXTRATO_CONTRATO,2,anotador_69243565,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.0,1.0,NaN
2,672,EXTRATO_CONTRATO,3,anotador_72589596,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.0,1.0,NaN
3,672,processo_gdf,5,anotador_69243565,53,15,001-000853/2019,0.0,1.0,1.0,NaN
4,672,processo_gdf,6,anotador_72589596,53,15,001-000853/2019,0.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2270,672,vigencia_contrato,2375,anotador_59684932,166035,46,12 (doze) meses a contar da data de assinatura,0.0,1.0,1.0,NaN
2271,672,vigencia_contrato,2376,anotador_69243565,166035,46,12 (doze) meses a contar da data de assinatura,0.0,1.0,1.0,NaN
2272,672,data_assinatura_contrato,2377,anotador_72589596,166103,10,25/07/2019,0.0,1.0,1.0,NaN
2273,672,data_assinatura_contrato,2378,anotador_59684932,166103,10,25/07/2019,0.0,1.0,1.0,NaN


In [10]:
df_with_metrics = pd.DataFrame(columns=dodf_dataframes[id_dodfs[0]].columns)
for df in dodf_dataframes.values():
    df_with_metrics = pd.concat([df_with_metrics, df])

In [11]:
# Trata valores nan
df_with_metrics.loc[df_with_metrics['edit_distance'].isnull(), 'edit_distance'] = \
    df_with_metrics[df_with_metrics['edit_distance'].isnull()]['length']

df_with_metrics.loc[df_with_metrics['jaccard_index'].isnull(), 'jaccard_index'] = 0
df_with_metrics.loc[df_with_metrics['cos_sim_tfidf'].isnull(), 'cos_sim_tfidf'] = 0
# df_with_metrics.loc[df_with_metrics['cos_sim_w2vec'].isnull(), 'cos_sim_w2vec'] = 0

In [12]:
csv_with_metrics = "df_with_metrics.csv"
df_with_metrics.to_csv(csv_with_metrics)
df_with_metrics

,id_dodf,tipo_ent,id_ent,anotador_ent,offset,length,texto,edit_distance,jaccard_index,cos_sim_tfidf,cos_sim_word2vec
0,672,EXTRATO_CONTRATO,1,anotador_59684932,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.00000,1.000000,NaN
1,672,EXTRATO_CONTRATO,2,anotador_69243565,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.00000,1.000000,NaN
2,672,EXTRATO_CONTRATO,3,anotador_72589596,23,1227,EXTRATO DE CONTRATO Processo: 001-000853/2019....,0.0,1.00000,1.000000,NaN
3,672,processo_gdf,5,anotador_69243565,53,15,001-000853/2019,0.0,1.00000,1.000000,NaN
4,672,processo_gdf,6,anotador_72589596,53,15,001-000853/2019,0.0,1.00000,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
27738,638,objeto_licitacao,3737,anotador_37219154,210322,367,Registro de Preços para contratação de empresa...,2.0,0.99458,1.000000,NaN
27739,638,orgao_licitante,3738,anotador_37219154,210631,58,Secretaria de Estado do Trabalho Distrito Fede...,0.0,1.00000,1.000000,NaN
27740,638,orgao_licitante,3739,anotador_27589498,210631,58,Secretaria de Estado do Trabalho Distrito Fede...,0.0,1.00000,1.000000,NaN
27741,638,nome_responsavel,3740,anotador_37219154,210768,25,JUNILMA OLIVEIRA FERREIRA,1.0,0.96000,0.371556,NaN


# Valor médio das métricas por tipo de entidade

In [13]:
# {
#   tipo_ent: {
#       edit_distance: {
#           mean: value,
#           std:  value
#       }
#       jaccard_index: {
#           mean: value,
#           std:  value
#       }
#       cos_sim_tfidf: {
#           mean: value,
#           std:  value
#       }
#       cos_sim_w2vec: {
#           mean: value,
#           std:  value
#       }
#   }
#   ...
# }
metrics_per_type = {}
for entity_type in df_with_metrics['tipo_ent'].unique():
    metrics_per_type[entity_type] = {}
    metrics_per_type[entity_type]['edit_distance'] = {}
    metrics_per_type[entity_type]['jaccard_index'] = {}
    metrics_per_type[entity_type]['cos_sim_tfidf'] = {}
    # metrics_per_type[entity_type]['cos_sim_w2vec'] = {}

    df_entity = df_with_metrics[df_with_metrics['tipo_ent'] == entity_type]
    metrics_per_type[entity_type]['edit_distance']['mean'] = np.mean(df_entity['edit_distance'].values)
    metrics_per_type[entity_type]['edit_distance']['std']  = np.std(df_entity['edit_distance'].values)
    metrics_per_type[entity_type]['jaccard_index']['mean'] = np.mean(df_entity['jaccard_index'].values)
    metrics_per_type[entity_type]['jaccard_index']['std']  = np.std(df_entity['jaccard_index'].values)
    metrics_per_type[entity_type]['cos_sim_tfidf']['mean'] = np.mean(df_entity['cos_sim_tfidf'].values)
    metrics_per_type[entity_type]['cos_sim_tfidf']['std']  = np.std(df_entity['cos_sim_tfidf'].values)
    # metrics_per_type[entity_type]['cos_sim_w2vec']['mean'] = np.mean(df_entity['cos_sim_w2vec'].values)
    # metrics_per_type[entity_type]['cos_sim_w2vec']['std']  = np.std(df_entity['cos_sim_w2vec'].values)

metrics_per_type

{'EXTRATO_CONTRATO': {'edit_distance': {'mean': 147.86977531548175,
   'std': 485.75061720928346},
  'jaccard_index': {'mean': 0.8912163084401672, 'std': 0.3041036198430798},
  'cos_sim_tfidf': {'mean': 0.8970431592485647, 'std': 0.30206541419070465}},
 'processo_gdf': {'edit_distance': {'mean': 1.2968612191958484,
   'std': 4.882615658983238},
  'jaccard_index': {'mean': 0.9369431620358325, 'std': 0.23429849925603094},
  'cos_sim_tfidf': {'mean': 0.9387333857139728, 'std': 0.2336695737429872}},
 'numero_contrato': {'edit_distance': {'mean': 1.9859822119102906,
   'std': 3.255894778340361},
  'jaccard_index': {'mean': 0.7846096363504178, 'std': 0.3472180101160544},
  'cos_sim_tfidf': {'mean': 0.7936016601109402, 'std': 0.34502924099619364}},
 'data_assinatura_contrato': {'edit_distance': {'mean': 1.0546831955922862,
   'std': 3.103655922920764},
  'jaccard_index': {'mean': 0.897211787294432, 'std': 0.30025422414294395},
  'cos_sim_tfidf': {'mean': 0.8969915168284384, 'std': 0.300831572

# Krinppendorff alpha

In [14]:
# Codificando labels de tipo de entidade para usar na matriz de entrada
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(df_with_metrics['tipo_ent'].values)
label_encoder.classes_

array(['AVISO_ANUL_REV_LICITACAO', 'AVISO_LICITACAO',
       'AVISO_SUSPENSAO_LICITACAO', 'EXTRATO_ADITAMENTO_CONTRATUAL',
       'EXTRATO_CONTRATO', 'EXTRATO_CONVENIO', 'cnpj_entidade_contratada',
       'cnpj_entidade_convenente', 'cnpj_orgao_concedente',
       'cnpj_orgao_contratante', 'codigo_licitacao_sistema_compras',
       'codigo_siggo', 'data_abertura_licitacao',
       'data_assinatura_contrato', 'data_assinatura_convenio',
       'data_escrito', 'decisao_tcdf', 'entidade_contratada',
       'entidade_convenente', 'fonte_recurso', 'identificacao_ocorrencia',
       'modalidade_licitacao', 'natureza_despesa', 'nome_responsavel',
       'nota_empenho', 'numero_contrato', 'numero_convenio',
       'numero_licitacao', 'numero_termo_aditivo',
       'objeto_aditamento_contratual', 'objeto_contrato',
       'objeto_convenio', 'objeto_licitacao', 'orgao_concedente',
       'orgao_contratante', 'orgao_licitante', 'processo_gdf',
       'programa_trabalho', 'sistema_compras', 'tipo_

In [15]:
# {
#   annotation_group: {
#       anotador_ent: label,
#       anotador_ent: label,
#       ...
#   }
#   ...
# }
dict_krippendorff = {}
for annotation_groups in dodf_annotation_groups.values():
    for group in annotation_groups:
        dict_krippendorff[group] = {}
        for anno_index in group:
            annotator = df_with_metrics.loc[anno_index]['anotador_ent']
            label = label_encoder.transform([df_with_metrics.loc[anno_index]['tipo_ent']])
            dict_krippendorff[group][annotator] = label[0]

dict_krippendorff

{frozenset({1217, 1218, 1219}): {'anotador_135115': 27,
  'anotador_26827896': 27,
  'anotador_59821738': 27},
 frozenset({1535, 1536, 1537}): {'anotador_26827896': 43,
  'anotador_59821738': 43,
  'anotador_135115': 43},
 frozenset({1930, 1931, 1932}): {'anotador_59821738': 38,
  'anotador_26827896': 38,
  'anotador_135115': 38},
 frozenset({1997, 1998, 1999}): {'anotador_59821738': 10,
  'anotador_26827896': 10,
  'anotador_135115': 10},
 frozenset({1102, 1103, 1104}): {'anotador_69243565': 4,
  'anotador_72589596': 25,
  'anotador_59684932': 25},
 frozenset({413, 414}): {'anotador_46084830': 25, 'anotador_37204582': 25},
 frozenset({552, 553, 554}): {'anotador_135115': 36,
  'anotador_59821738': 36,
  'anotador_26827896': 36},
 frozenset({277, 278}): {'anotador_26827896': 1, 'anotador_59821738': 1},
 frozenset({999, 1000}): {'anotador_26827896': 36, 'anotador_135115': 36},
 frozenset({1069}): {'anotador_26827896': 35},
 frozenset({517, 518}): {'anotador_37204582': 34, 'anotador_4608

In [16]:
df_krippendorff = pd.DataFrame.from_dict(dict_krippendorff)
df_krippendorff

,"(1217, 1218, 1219)","(1536, 1537, 1535)","(1930, 1931, 1932)","(1997, 1998, 1999)","(1104, 1102, 1103)","(413, 414)","(552, 553, 554)","(277, 278)","(1000, 999)",(1069),...,(27493),"(27593, 27594)","(26394, 26395)","(26153, 26154, 26155, 26156)",(25801),"(25883, 25884)","(24968, 24967)","(27184, 27185, 27183)","(27365, 27366, 27367)","(26436, 26437, 26438)"
anotador_135115,27.0,43.0,38.0,10.0,NaN,NaN,36.0,NaN,36.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anotador_26827896,27.0,43.0,38.0,10.0,NaN,NaN,36.0,1.0,36.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anotador_59821738,27.0,43.0,38.0,10.0,NaN,NaN,36.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN
anotador_69243565,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,22.0,41.0
anotador_72589596,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,22.0,41.0
anotador_59684932,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,22.0,41.0
anotador_46084830,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,...,29.0,34.0,25.0,25.0,NaN,34.0,NaN,NaN,NaN,NaN
anotador_37204582,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,...,NaN,34.0,25.0,25.0,NaN,34.0,NaN,NaN,NaN,NaN
anotador_37219154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN
anotador_27589498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN


In [17]:
import krippendorff
alpha = krippendorff.alpha(reliability_data=df_krippendorff, level_of_measurement='nominal')
alpha

0.9730289497472199

# Número de anotações por tipo de entidade

In [29]:
df_with_metrics['tipo_ent'].value_counts()

processo_gdf                        2570
orgao_contratante                   1972
numero_contrato                     1724
nome_responsavel                    1183
numero_licitacao                    1104
entidade_contratada                 1084
EXTRATO_CONTRATO                    1083
objeto_contrato                     1039
objeto_licitacao                     951
vigencia_contrato                    938
sistema_compras                      900
AVISO_LICITACAO                      873
tipo_objeto                          873
modalidade_licitacao                 830
fonte_recurso                        801
valor_contrato                       800
orgao_licitante                      774
data_abertura_licitacao              758
data_assinatura_contrato             726
programa_trabalho                    712
nota_empenho                         680
numero_termo_aditivo                 628
codigo_licitacao_sistema_compras     611
EXTRATO_ADITAMENTO_CONTRATUAL        607
objeto_aditament

# Escreve as linhas da tabela de resultados para o relatório

In [36]:
import re
for entity_type in df_with_metrics['tipo_ent'].unique():
    escaped_entity_type = re.sub(r'_', r'\\_', entity_type)
    edit_distance_mean  = f"{metrics_per_type[entity_type]['edit_distance']['mean']:.0f}"
    edit_distance_std   = f"{metrics_per_type[entity_type]['edit_distance']['std']:.0f}"
    jaccard_index_mean  = f"{metrics_per_type[entity_type]['jaccard_index']['mean']:.1f}"
    jaccard_index_std   = f"{metrics_per_type[entity_type]['jaccard_index']['std']:.1f}"
    cos_sim_tfidf_mean  = f"{metrics_per_type[entity_type]['cos_sim_tfidf']['mean']:.1f}"
    cos_sim_tfidf_std   = f"{metrics_per_type[entity_type]['cos_sim_tfidf']['std']:.1f}"
    num_annotations     = df_with_metrics['tipo_ent'].value_counts()[entity_type]
    print(f"{escaped_entity_type:<36}"
          f"& $ {edit_distance_mean:<4}\\pm {edit_distance_std:<4}$      "
          f"& $ {jaccard_index_mean:<4}\\pm {jaccard_index_std:<4}$  "
          f"& $ {cos_sim_tfidf_mean:<4}\\pm {cos_sim_tfidf_std:<4}$ "
          f"& $ {num_annotations:<5}$   \\\\ \\hline")

EXTRATO\_CONTRATO                   & $ 148 \pm 486 $      & $ 0.9 \pm 0.3 $  & $ 0.9 \pm 0.3 $ & $ 1083 $   \\ \hline
processo\_gdf                       & $ 1   \pm 5   $      & $ 0.9 \pm 0.2 $  & $ 0.9 \pm 0.2 $ & $ 2570 $   \\ \hline
numero\_contrato                    & $ 2   \pm 3   $      & $ 0.8 \pm 0.3 $  & $ 0.8 \pm 0.3 $ & $ 1724 $   \\ \hline
data\_assinatura\_contrato          & $ 1   \pm 3   $      & $ 0.9 \pm 0.3 $  & $ 0.9 \pm 0.3 $ & $ 726  $   \\ \hline
orgao\_contratante                  & $ 8   \pm 17  $      & $ 0.7 \pm 0.4 $  & $ 0.7 \pm 0.4 $ & $ 1972 $   \\ \hline
entidade\_contratada                & $ 6   \pm 13  $      & $ 0.8 \pm 0.4 $  & $ 0.8 \pm 0.3 $ & $ 1084 $   \\ \hline
cnpj\_entidade\_contratada          & $ 1   \pm 4   $      & $ 0.9 \pm 0.2 $  & $ 0.9 \pm 0.2 $ & $ 225  $   \\ \hline
objeto\_contrato                    & $ 38  \pm 125 $      & $ 0.9 \pm 0.3 $  & $ 0.9 \pm 0.3 $ & $ 1039 $   \\ \hline
valor\_contrato                     & $ 1   \pm 